## 축소전 데이터와 PCA 이후 데이터 Training 시간 비교

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')


In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.shape

(42000, 785)

In [4]:
label = data.iloc[:,0]
X = data.iloc[:,1:]
#X = StandardScaler().fit_transform(X)

In [5]:
data.shape

(42000, 785)

In [6]:
X_tr, X_test, y_tr, y_test = train_test_split(X, label, test_size = 0.2, random_state = 4)

In [7]:
X_tr.shape, y_tr.shape

((33600, 784), (33600,))

In [8]:
X_test.shape, y_test.shape

((8400, 784), (8400,))

### 1) 축소 전 데이터 KNN

In [9]:
knn = KNeighborsClassifier(n_neighbors=5)
# Train 
time_start = time.time()
knn.fit(X_tr,y_tr)
print('knn done! Time elapsed: {} seconds'.format(time.time()-time_start))

knn done! Time elapsed: 7.6910271644592285 seconds


In [10]:
time_start = time.time()
y_pred = knn.predict(X_test)
print('knn done! Time elapsed: {} seconds'.format(time.time()-time_start))
# test accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))


knn done! Time elapsed: 420.91319704055786 seconds
Accuracy: 0.965


### 2) 축소 전 데이터 Randomforest

In [11]:
#Create a Randomforest
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

time_start = time.time()
#Train
rf.fit(X_tr, y_tr)
print('randomforest done! Time elapsed: {} seconds'.format(time.time()-time_start))

randomforest done! Time elapsed: 239.9919981956482 seconds


In [12]:
#Predict the response for test dataset

time_start = time.time()
y_pred = rf.predict(X_test)
print('randomforest done! Time elapsed: {} seconds'.format(time.time()-time_start))
# test accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))


randomforest done! Time elapsed: 3.943236827850342 seconds
Accuracy: 0.9647619047619047


### PCA 로 차원축소

In [16]:
pca = PCA(2)

# 트레인으로 일단 차원축소하고 Train 데이터 기준으로 test 데이터도 차원축소해줌
# train set의 mean과 std로 test set을 정규화해주는 것과 같은 이치
pca.fit(X_tr)

X_train_pca = pca.transform(X_tr)
X_test_pca = pca.transform(X_test)

### 1) 축소 후 데이터 KNN

In [17]:
knn = KNeighborsClassifier(n_neighbors=5)
# Train 
time_start = time.time()
knn.fit(X_train_pca,y_tr)
print('knn done! Time elapsed: {} seconds'.format(time.time()-time_start))

knn done! Time elapsed: 0.023234128952026367 seconds


In [18]:
time_start = time.time()
y_pred = knn.predict(X_test_pca)
print('knn done! Time elapsed: {} seconds'.format(time.time()-time_start))
# test accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))


knn done! Time elapsed: 0.03829002380371094 seconds
Accuracy: 0.41


### 2) 축소 후 데이터 SVM

In [19]:
#Create a svm Classifier
rf = RandomForestClassifier(n_estimators=1000, random_state=22)

time_start = time.time()
#Train
rf.fit(X_train_pca, y_tr)
print('randomforest done! Time elapsed: {} seconds'.format(time.time()-time_start))

randomforest done! Time elapsed: 65.084153175354 seconds


In [20]:
#Predict the response for test dataset

time_start = time.time()
y_pred = rf.predict(X_test_pca)
print('randomforest done! Time elapsed: {} seconds'.format(time.time()-time_start))
# test accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))


randomforest done! Time elapsed: 8.280842065811157 seconds
Accuracy: 0.4064285714285714


## 성능이 떨어졌으므로, RandomSearch나 GridSearch로 최적의 k(차원값)을 찾아야함